In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import re
import h5py
from six.moves import range

from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
%matplotlib inline

/Users/perdue/miniconda2/envs/py2mnv_pda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import sys
sys.path.append('/Users/perdue/Documents/MINERvA/AI/ANNMINERvA/')

from evtid_utils import decode_eventid
from mnvtf.reader_sqlite import MnvCategoricalSQLiteReader
from mnvtf.reader_text import MnvCategoricalTextReader

In [3]:
from mnvdlplot.conf_mat_pl import make_conf_mat_plots_rowcolnormonly
from mnvdlplot.conf_mat_pl import make_conf_mat_plots_raw

In [4]:
h5path = '/Users/perdue/Documents/MINERvA/AI/hdf5/201801/'
h5name = h5path + 'hadmultkineimgs_127x94_me1Pmc.hdf5'
h5file = h5py.File(h5name, 'r')

In [6]:
dbpath = '/Users/perdue/Documents/MINERvA/AI/minerva_tf/predictions/201801/'
dbname = dbpath + 'mnv_st_epsilon_predictionsME1LOPMC_VALIDCHECK_model_20180323_def_chngdpad_planecodes_nclass173_trainME1LOPMC_validME1LOPMC_testME1LOPMC_optADAM_batchsz1024_nodoBatchNorm_tev.txt.gz'
n_classes = 173
reader_mc = MnvCategoricalTextReader(dbname)

In [37]:
mcgen = reader_mc.read_data_generator()
n_events = 1

In [59]:
for i, d in enumerate(mcgen):
    evtid = int(d[0])
    pred_seq = d[1]
    probs_list = d[2]
    found_idx = np.where(h5file['event_data']['eventids'][:] == evtid)[0]
    if len(found_idx):
        found_idx = found_idx[0]
        print('{}, {}, {}'.format(
            evtid, pred_seq, h5file['vtx_data']['planecodes'][found_idx][0]
        ))
        break
#    if i > n_events:
#        break

1120070038042501, 164, 164
